In [1]:
import torch
import schnetpack as spk
import math


import numpy as np
from numpy import savetxt

In [2]:
def print_molecule(props):
    
    # load x,y,z coordinates tensors
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            print('H',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 6:
            print('C',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 7:
            print('N',x[i],y[i],z[i])     
        if props['_atomic_numbers'][i] == 8:
            print('O',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 9:
            print('F',x[i],y[i],z[i]) 
    print('')  
    return z

In [3]:
def hook_layer(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global layer_output
    #Update the embedding_output variable to be equal to our output tensor
    layer_output=out_tensor 

In [4]:
def convert_2D(number_of_atoms,layer_output):
    layer = np.zeros((number_of_atoms,30))
    for i in range(number_of_atoms):
        for j in range(30):
            layer[i][j] = layer_output[0][i][j]
    return layer

In [ ]:
# Load QM9 dataset
from schnetpack.datasets import QM9

qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)
# Load split file 
train, val, test = spk.data.train_test_split(qm9data,split_file='../../../data/trained_models/qm9_i3_30f_10000_5000/split.npz')

# Load atom ref data 
atomrefs = qm9data.get_atomref(QM9.U0)
# Define SchNet representation model

schnet = spk.representation.SchNet(
    n_atom_basis=30, n_filters=30, n_gaussians=20, n_interactions=3,
    cutoff=4. , cutoff_network=spk.nn.cutoff.CosineCutoff
)

# Define SchNet output model and property to be predicted

output_U0 = spk.atomistic.Atomwise(n_in=30, atomref=atomrefs[QM9.U0])

# Define atomistic model

model = spk.AtomisticModel(representation=schnet,output_modules=output_U0)

# Load saved checkpoint file
checkpoint_path = '../../../data/trained_models/qm9_i3_30f_10000_5000/trained.pth'
load_checkpoint = torch.load(checkpoint_path)

#qm9_i6_30f_20g-1000-500-4_300.pth
# load model's state dictionary from saved checkpoint
model.load_state_dict(load_checkpoint)


number_of_inputs = 5000

#set up device for forward pass
device='cpu'

# load atoms converter 
converter = spk.data.AtomsConverter(device=device)

# load spk calculator
calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.U0)

data = np.zeros((1,30))
datahae = np.zeros((1))
dataoae = np.zeros((1))
datah = np.zeros((1,30))
for idx in range(number_of_inputs):
        
    # load data for molecule
    at, props = qm9data.get_properties(idx)
        
    # print molecule for identification
    print(idx)
    z = print_molecule(props)
    number_of_atoms=len(z)
        
    # set calculator on molecule
    at.set_calculator(calculator)
        
    # convert qm9 data to machine-readable form
    inputs = converter(at)
        
    #Instatiate layer output
    layer_output=None
        
    # Forward hook the model's interaction layer 
    model.representation.interactions[0].filter_network[0].register_forward_hook(hook_layer)
        
    # Forward pass molecules through the model
    model(inputs)
    
    layer_output = layer_output.detach().numpy()
    
    from schnetpack.atomistic.output_modules import yi
    
    yi=yi.detach().numpy()
    
    print(layer_output)

    #convert layer tensor to 2D array
    rows = convert_2D(number_of_atoms,layer_output)

    #save the vector of every oxygen atom encountered
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            data = np.vstack((data,rows[i]))
            
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datah = np.vstack((datah,rows[i]))    
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datahae = np.vstack((datahae,yi[0][i]))
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            dataoae = np.vstack((dataoae,yi[0][i]))            
    
print('DONE')            
savetxt('../../../data/dataO.csv',data,delimiter=',')
savetxt('../../../data/dataH.csv',datah,delimiter=',')
savetxt('../../../data/hae.csv',datahae,delimiter=',')
savetxt('../../../data/oae.csv',dataoae,delimiter=',')

# Filter network

0
C -2.8340169e-06 2.3049886e-06 -1.4378233e-07
H 0.014845718 -1.0918331 -0.0060250196
H 1.0244261 0.3779494 -0.007724565
H -0.52811974 0.36172476 -0.88464487
H -0.5111183 0.3521308 0.89839613

[-2.34545559e-01  1.08992159e-01 -2.21908599e-01  2.94135213e-02
 -2.14937329e-02  1.93321705e-03 -1.62687957e-01  1.10212624e-01
  1.08757794e-01 -6.33331537e-02 -1.33893609e-01 -2.38537788e-04
 -1.08166635e-01 -4.86618876e-02  4.03444171e-02 -4.75584865e-02
  2.19644547e-01 -1.61880553e-01 -1.35993600e-01  1.72948241e-02
 -9.17410851e-02  6.40261173e-03 -1.59299850e-01  2.94629753e-01
  2.27828026e-02 -2.54097819e-01  6.12241626e-02  5.82536459e-02
  7.95367360e-02  9.32472944e-03]
[-2.3454714e-01  1.0899395e-01 -2.2190624e-01  2.9420853e-02
 -2.1499217e-02  1.9325614e-03 -1.6268629e-01  1.1021042e-01
  1.0875976e-01 -6.3332736e-02 -1.3389683e-01 -2.3543835e-04
 -1.0816491e-01 -4.8661470e-02  4.0344477e-02 -4.7561765e-02
  2.1964127e-01 -1.6187990e-01 -1.3599449e-01  1.7294586e-02
 -9.1749370e

[-0.23292068  0.10711348 -0.22425708  0.02206415 -0.01597887  0.0025785
 -0.16429889  0.11238611  0.10675275 -0.06378555 -0.13054162 -0.00329852
 -0.10979825 -0.04904556  0.04030621 -0.04421169  0.22289157 -0.16252333
 -0.13512552  0.01752037 -0.08328134  0.00528342 -0.16152793  0.29269886
  0.02677125 -0.2536534   0.05786842  0.05961728  0.0802542   0.00987697]
[-0.2329207   0.10711348 -0.22425702  0.02206433 -0.01597905  0.0025785
 -0.16429883  0.11238605  0.10675275 -0.06378549 -0.13054168 -0.0032984
 -0.10979825 -0.04904556  0.04030621 -0.04421169  0.22289151 -0.16252333
 -0.13512552  0.01752037 -0.08328158  0.00528347 -0.16152787  0.29269892
  0.02677113 -0.25365344  0.05786848  0.05961722  0.0802542   0.00987697]
[ 0.08459133 -0.11679071 -0.08811837 -0.08207566  0.26159877 -0.01139235
  0.12473977 -0.09125686 -0.05697584  0.00081843  0.07404453  0.05454093
 -0.04263592  0.01350504 -0.07800364 -0.04645181 -0.11285019  0.02276284
  0.16096199 -0.01506686  0.04813951 -0.00702971  0.


[-0.23499563  0.10949832 -0.22123727  0.03151584 -0.02304208  0.00174111
 -0.16220516  0.1095807   0.10931224 -0.06320137 -0.13482523  0.00061876
 -0.10768545 -0.04855025  0.04035401 -0.04849929  0.21869385 -0.16168064
 -0.13623863  0.0172286  -0.09412289  0.00673604 -0.15863985  0.29513955
  0.02164412 -0.25421575  0.06217247  0.05789602  0.07931936  0.00915605]
[-0.23330864  0.10756904 -0.22370678  0.02378541 -0.01728493  0.00243109
 -0.16393268  0.11188221  0.1072318  -0.06368089 -0.13134027 -0.00257295
 -0.10942316 -0.04895663  0.04031569 -0.04500383  0.22214204 -0.16238093
 -0.13533044  0.01746845 -0.08528155  0.00553894 -0.16101736  0.2931723
  0.02583611 -0.253762    0.05866003  0.05928171  0.08009237  0.00975293]
[ 0.3091396   0.0114677  -0.13919723 -0.12848997 -0.05016863 -0.15707207
 -0.0520159   0.08742648 -0.04142624 -0.01017326  0.19669509 -0.17426687
  0.39550006  0.13510406 -0.21961275  0.17320323 -0.19620493  0.16583347
  0.10558814 -0.06140012 -0.16215873 -0.07002378 

C 0.16795558 1.4243842 -0.31935304
C 0.19002895 -0.10485057 -0.29820374
C 0.91049814 -0.6594086 0.9248196
O -1.1310841 -0.64056706 -0.26260433
H 1.1832712 1.8337176 -0.34678715
H -0.36276668 1.7974348 -1.2034236
H -0.3397112 1.809968 0.57079
H 0.7115355 -0.46111393 -1.2031567
H 0.42351922 -0.3113001 1.8415376
H 0.88353455 -1.7523178 0.9180865
H 1.9554573 -0.335911 0.940277
H -1.6170725 -0.27921587 -1.0105046

[-0.23314288  0.10737497 -0.22394267  0.02304757 -0.0167262   0.00249463
 -0.16409045  0.11209863  0.10702711 -0.06372589 -0.13099885 -0.00288337
 -0.10958451 -0.04899484  0.04031163 -0.04466486  0.2224642  -0.16244262
 -0.1352427   0.0174908  -0.08442557  0.00542891 -0.16123712  0.29297096
  0.02623695 -0.25371584  0.05832112  0.05942428  0.08016223  0.00980657]
[-0.23365569  0.10797292 -0.22320911  0.0253427  -0.01845884  0.00229579
 -0.16359538  0.11142361  0.10766017 -0.06358552 -0.13205546 -0.00192124
 -0.10908008 -0.04887569  0.04032403 -0.04571605  0.22145814 -0.16224772
 -

26
O -0.83661675 -1.3895576 0.0010744806
C -0.8137457 -0.18963368 0.005869432
C 0.4370984 0.5822472 -0.0011569314
N 1.4025168 1.2189847 -0.0064238496
H -1.7223324 0.43802574 0.01605794

27
O -0.6208464 -1.6154674 -0.0022703465
C -0.6413494 -0.4131876 0.0029220288
C 0.64134026 0.41319242 -0.0039194413
O 0.62085307 1.6154643 0.0029703435
H -1.5711225 0.19277982 0.014684269
H 1.5711266 -0.19278784 -0.013909801

[ 0.06050831 -0.10588145 -0.08408439 -0.06451327  0.2542351  -0.01129836
  0.11860442 -0.1074872  -0.03706616 -0.02553725  0.04547113  0.07440048
 -0.00115538  0.02750736 -0.09471691 -0.08598632 -0.14472133 -0.01333958
  0.21054357 -0.00680953  0.00972295 -0.01168358  0.03335911 -0.2651034
  0.03770065  0.4701277  -0.06193119 -0.04571599 -0.0612942   0.00181729]
[-0.22892392  0.10219979 -0.2295681   0.00547087 -0.00291812  0.00387383
 -0.16744572  0.11706084  0.10182208 -0.06473893 -0.12238395 -0.01057792
 -0.11319226 -0.04986346  0.04021049 -0.03629208  0.2297349  -0.16361612
 -0.

...

[ 0.12582988 -0.11284041 -0.09304541 -0.09436595  0.25125313 -0.00714016
  0.11709297 -0.05375415 -0.0913201   0.04920989  0.1023587   0.02558899
 -0.09873694 -0.01633328 -0.05626404  0.01409847 -0.04936576  0.08138424
  0.09090632 -0.01977527  0.12385517  0.00334388  0.06057352 -0.19637677
 -0.02580458  0.50985587  0.04852152 -0.08942819 -0.00607574  0.00399035]
[ 0.31692445 -0.09425956 -0.0921312  -0.23441744  0.06790614 -0.07700586
 -0.15724725  0.04596967 -0.12216812  0.15203005  0.34043372 -0.13389152
  0.5901835   0.35362053  0.01444083  0.3051613  -0.06250274  0.19793236
  0.05676019 -0.08643019 -0.10433251 -0.0195055   0.03923881 -0.1703257
 -0.2143398  -0.4049194  -0.15279675  0.01453954 -0.3870105  -0.04816657]
[ 3.1353748e-01  2.3026466e-03 -1.3906723e-01 -1.4077502e-01
 -4.5203865e-02 -1.5417510e-01 -5.7156503e-02  8.6818993e-02
 -4.4135988e-02 -3.9899349e-04  2.1176594e-01 -1.7497438e-01
  4.0565979e-01  1.5580052e-01 -2.0678568e-01  1.8841928e-01
 -1.9036746e-01  1.733828

92
C -1.111202 1.4803663 0.13194007
C -0.66964763 0.031853613 0.10723244
C 0.086773075 -0.49478114 -1.1347536
C 1.1902907 -0.8637229 -0.12493972
O 0.48532954 -0.21133919 0.9504042
H -0.2959999 2.1394398 -0.18281281
H -1.9642781 1.6320868 -0.5388842
H -1.4158894 1.770331 1.1425959
H -1.5022218 -0.62104 0.4084369
H -0.3777866 -1.3198452 -1.6766365
H 0.3674883 0.2888146 -1.8421314
H 1.3068902 -1.9406457 0.056403007
H 2.1815836 -0.42638958 -0.29161626

[-0.23337457  0.10764605 -0.22361267  0.02407998 -0.01750749  0.00240564
 -0.16386932  0.11179566  0.10731316 -0.06366289 -0.13147604 -0.00244933
 -0.10935855 -0.04894137  0.0403173  -0.0451389   0.22201312 -0.16235602
 -0.13536543  0.01745957 -0.08562267  0.00558311 -0.16092938  0.293252
  0.02567613 -0.25378036  0.05879509  0.05922526  0.08006424  0.00973141]
[-0.23371384  0.10804021 -0.22312516  0.0256052  -0.01865608  0.00227284
 -0.16353804  0.11134601  0.10773188 -0.06356937 -0.13217545 -0.0018118
 -0.10902184 -0.04886198  0.04032546 -


[-0.23369107  0.10801393 -0.223158    0.02550256 -0.01857901  0.00228179
 -0.16356051  0.11137635  0.10770386 -0.06357563 -0.13212854 -0.00185454
 -0.10904467 -0.04886734  0.04032487 -0.04578894  0.22138757 -0.16223377
 -0.13553387  0.01741618 -0.08726537  0.00579786 -0.16050118  0.29363155
  0.02490383 -0.2538676   0.05944628  0.05895686  0.07992703  0.00962597]
[ 0.03335357 -0.07277185 -0.07746238 -0.03076273  0.23120743 -0.01004618
  0.09363341 -0.12140346 -0.01509696 -0.05724424 -0.0017007   0.10154152
  0.06734031  0.03613335 -0.12304407 -0.13838285 -0.17746937 -0.05930746
  0.2864123   0.00954926 -0.02939761 -0.01509547  0.02014178 -0.2783673
  0.09109658  0.3918854  -0.10602325 -0.04091126 -0.05769253  0.00208008]
[ 0.07856977 -0.11510414 -0.08720982 -0.07843566  0.26063192 -0.01150692
  0.12410313 -0.09564525 -0.05198032 -0.00579548  0.06774378  0.05925578
 -0.03315997  0.01732445 -0.08177906 -0.05598575 -0.12113255  0.01394385
  0.17250037 -0.01340151  0.03815466 -0.00830567 

  0.01946288 -0.2544311   0.06397772  0.05724919  0.07888818  0.00881952]
[ 4.71521020e-02 -9.37430263e-02 -8.12431574e-02 -5.05907536e-02
  2.45604932e-01 -1.07321739e-02  1.09889150e-01 -1.14991546e-01
 -2.61583328e-02 -4.04834151e-02  2.50678062e-02  8.68595243e-02
  2.78273821e-02  3.32450271e-02 -1.06617332e-01 -1.10103130e-01
 -1.61116362e-01 -3.47870588e-02  2.43625998e-01 -6.07371330e-05
 -9.95469093e-03 -1.36681795e-02  2.67973542e-02 -2.72528648e-01
  6.01080060e-02  4.39668536e-01 -8.27757120e-02 -4.13367152e-02
 -6.28376603e-02  1.94281340e-03]
[ 0.05889404 -0.10469908 -0.08376962 -0.06302023  0.25338948 -0.01124507
  0.1178     -0.1084519  -0.03574073 -0.02731472  0.04320765  0.07584023
  0.002047    0.02830023 -0.09602451 -0.08879423 -0.14674562 -0.01585913
  0.21426547 -0.00609028  0.00728744 -0.01194906  0.03257084 -0.26609322
  0.04014283  0.46700966 -0.06442362 -0.04505432 -0.06171656  0.00182831]
[ 0.07454044 -0.11361724 -0.08656812 -0.07573527  0.25966662 -0.0115284

  0.02526635 -0.25382686  0.05914086  0.05908209  0.07999182  0.00967574]
[-0.23334014  0.10760581 -0.22366184  0.02392602 -0.01739126  0.00241899
 -0.16390246  0.1118409   0.10727066 -0.06367224 -0.13140506 -0.00251389
 -0.10939234 -0.04894936  0.04031646 -0.04506826  0.22208053 -0.16236907
 -0.13534713  0.01746422 -0.08544439  0.00555998 -0.16097534  0.2932104
  0.0257597  -0.2537708   0.05872452  0.05925471  0.08007896  0.00974268]
[ 0.08082545 -0.11580914 -0.08755696 -0.07985383  0.2610607  -0.01147592
  0.12440348 -0.09402829 -0.05385065 -0.00332361  0.07016647  0.05747223
 -0.03676623  0.01591462 -0.0803389  -0.05239075 -0.118056    0.01726019
  0.16812295 -0.01405454  0.04186434 -0.00783587  0.04299438 -0.25030622
  0.01152509  0.49927342 -0.03034633 -0.05621994 -0.05166554  0.001849  ]
[-7.7456832e-03 -5.7148933e-04 -5.6304574e-02 -3.4589231e-02
  1.0380527e+00  1.0378222e+00 -7.0547819e-02 -2.0397490e-01
 -3.7063432e-01 -4.1815579e-02  4.5229363e-01 -3.7605762e-02
  1.4839325e